In [ ]:
# 1. convert from many txt files to one csv file

import os
from tqdm import tqdm
import csv

txts_path = "./save"

txt_paths = []
for root, dirs, files in os.walk(txts_path):
    for file in files:
        txt_paths.append(root + "/" + file)

# 2022 data
all = []
for path in tqdm(txt_paths):
    with open(path, "r", encoding='utf-8') as f:
        data = f.readlines()
        one = {}
        one['time'] = data[5][0:10]
        one['title'] = data[1].replace('\n', '')
        one['content'] = "".join(data[5:]).replace('\n', '').replace('\u3000\u3000', '')[10:]
        all.append(one)

# # 2018 data
# all = []
# for path in tqdm(txt_paths):
#     with open(path, "r", encoding='utf-8') as f:
#         data = f.read()
#         data_list = data.split('&')
#         one = {}
#         one['time'] = data_list[0]
#         one['title'] = data_list[1]
#         one['content'] = "".join(data_list[3:]).replace('\n', '')
#         all.append(one)

with open("./save/eastmoney_2022.csv","w+", newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    for one in all:
        temp = [one['time'], one['title'], one['content']]
        writer.writerow(temp)

In [ ]:
# 2. use baidu_api to analyze

import pandas as pd
from tqdm import tqdm
from utility.baidu_utils import BaiduNLU
import pickle


data = pd.read_csv("./save/eastmoney_2022.csv", header=None)
print(len(data))

config = {}
config['APP_ID'] = '25576077'
config['API_KEY'] = 'iprdevAUKAE15tQkgEYpCyYQ'
config['SECRET_KEY'] = '***********************'
nlu_tool = BaiduNLU(config)

all = []
for i in tqdm(range(len(data))):
    line = data.iloc[i]
    title = line[1]
    content = line[2]

    try:
        res_sentiment = nlu_tool.get_sentiment(content)
    except Exception:
        res_sentiment = {}

    try:
        res_keyword = nlu_tool.get_keyword(title, content)
    except Exception:
        res_keyword = {}

    try:
        res_topic = nlu_tool.get_topic(title, content)
    except Exception:
        res_topic = {}

    try:
        res_summary = nlu_tool.get_summary(title, content, maxSummaryLen=100)
    except Exception:
        res_summary = {}

    one = {
        "time": line[0],
        "title": line[1],
        "content": line[2],
        "sentiment": res_sentiment["items"] if "items" in res_sentiment.keys() else None,
        "keyword": res_keyword["items"] if "items" in res_keyword.keys() else None,
        "topic": res_topic["item"] if "item" in res_topic.keys() else None,
        "summary": res_summary["summary"] if "summary" in res_summary.keys() else None,
    }
    all.append(one)


with open("./save/eastmoney.pkl", "wb") as f:
    pickle.dump(all, f)

In [ ]:
# 3. push to mongodb